In [ ]:
import os
from scipy.io.wavfile import read
import numpy as np
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.activations import selu
import keras
import tensorflow as tf
from keras import layers
from keras.layers import LeakyReLU
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.optimizers import SGD
from keras.layers.convolutional import MaxPooling1D
from keras.layers.convolutional import AveragePooling1D
from keras.layers import GlobalMaxPool1D
from keras.layers import GlobalAveragePooling1D
from keras.utils import to_categorical
from keras.regularizers import l2
from matplotlib import pyplot
from scipy.fft import fft
from tensorflow.keras.constraints import max_norm
from keras.layers import GaussianNoise 
from keras.layers import GaussianDropout 
from keras.layers import SpatialDropout1D 
from keras.layers import Concatenate 
from keras import Model
from keras import Input
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/My Drive/development.zip" -d "/content/"
!unzip "/content/drive/My Drive/evaluation.zip" -d "/content/"

Archive:  /content/drive/My Drive/development.zip
replace /content/development/i/482fcf4448c5804f492cc8a47384128ee1731a5f158bbbb6ddadefb93809cc7d.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  /content/drive/My Drive/evaluation.zip
replace /content/evaluation/8e123b8a4017cf5ab284b3f6eef325e974528950f1f318dc819f59f7a38bfd50.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
audios = []
time = []
labels = []
letters = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
n = 0

for lbl in letters:
    file = os.listdir('/content/development/' + lbl)
    for name in file:
        lung = len(np.array(read('/content/development/' + lbl + "/" + name)[1]))
        if 11999 == lung:
          appo = np.array(read('/content/development/' + lbl + "/" + name)[1])
          time.append(appo)
          audios.append(np.abs(fft(appo)))
          #audios.append(appo)
          labels.append(n)
        elif lung > 11999:
          arr = np.array(read('/content/development/' + lbl + "/" + name)[1])
          time.append(arr[:-1])
          audios.append(np.abs(fft(arr[:-1])))
          #audios.append(appo)
          labels.append(n)
    n+=1

In [ ]:
audios = np.array(audios)
time = np.array(time)
y = to_categorical(np.array(labels), 10)

In [ ]:
ss1 = StandardScaler()
sst1 = StandardScaler()
#ss1 = RobustScaler()
#sst1 = RobustScaler()
X = ss1.fit_transform(audios)
X = X.reshape((X.shape[0], 11999, 1))
X_t = sst1.fit_transform(time)
X_t = X_t.reshape((X_t.shape[0], 11999, 1))
audios = []
time = []

##Rete

In [ ]:
def create_model(n_features):
  model = Sequential()
  model.add(Conv1D(filters=128, kernel_size=3, strides = 1,  kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005), input_shape=(n_features, 1)))
  model.add(BatchNormalization())
  model.add(layers.Activation(relu))
  model.add(MaxPooling1D(pool_size=2))
  #if tempo == True:
   # model.add(GaussianNoise(0.2))
  #model.add(SpatialDropout1D(0.3))
  model.add(Conv1D(filters=192, kernel_size=3, strides = 2, kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
  model.add(BatchNormalization())
  model.add(layers.Activation(relu))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=192, kernel_size=3, strides = 2, kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
  model.add(BatchNormalization())
  model.add(layers.Activation(relu))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=256, kernel_size=5, strides = 2, kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
  model.add(BatchNormalization())
  model.add(layers.Activation(relu))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=392, kernel_size=5, strides = 2, kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
  model.add(BatchNormalization())
  model.add(layers.Activation(relu))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=512, kernel_size=7, strides = 2, kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
  model.add(BatchNormalization())
  model.add(layers.Activation(relu))
  #model.add(GlobalAveragePooling1D())
  model.add(MaxPooling1D())
  model.add(Flatten())
  #model.add(Dense(100, activation = "relu", kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005)))
  return model
  #model.summary()

In [ ]:
verbose, epochs, batch_size, n_features, n_outputs = 1, 11, 32, 11999, 10
time = create_model(n_features)
fourier = create_model(n_features)

# combine the output of the two branches
combined = keras.layers.Concatenate(axis=1)([time.output, fourier.output])

dense = Dense(100, activation = "relu", kernel_regularizer=l2(0.0005), bias_regularizer=l2(0.0005))(combined)
drop = Dropout(0.5)(dense)
out = Dense(n_outputs, activation='softmax')(drop)

model = Model(inputs=[fourier.input, time.input], outputs=out)
sgd = SGD(lr=0.001, momentum=0.9, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv1d_input (InputLayer)       [(None, 11999, 1)]   0                                            
__________________________________________________________________________________________________
conv1d_6_input (InputLayer)     [(None, 11999, 1)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 11997, 128)   512         conv1d_input[0][0]               
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 11997, 128)   512         conv1d_6_input[0][0]             
_______________________________________________________________________________________

In [ ]:
history = model.fit([X, X_t], y, epochs=epochs, validation_split=0, batch_size=batch_size, verbose=verbose)

Epoch 1/11
764/764 [==============================] - 89s 117ms/step - loss: 2.4816 - accuracy: 0.6312
Epoch 2/11
764/764 [==============================] - 89s 116ms/step - loss: 1.8108 - accuracy: 0.8618
Epoch 3/11
764/764 [==============================] - 89s 116ms/step - loss: 1.6379 - accuracy: 0.9169
Epoch 4/11
764/764 [==============================] - 89s 116ms/step - loss: 1.5453 - accuracy: 0.9407
Epoch 5/11
764/764 [==============================] - 89s 116ms/step - loss: 1.4852 - accuracy: 0.9541
Epoch 6/11
764/764 [==============================] - 89s 116ms/step - loss: 1.4412 - accuracy: 0.9618
Epoch 7/11
764/764 [==============================] - 89s 116ms/step - loss: 1.4009 - accuracy: 0.9677
Epoch 8/11
764/764 [==============================] - 89s 116ms/step - loss: 1.3666 - accuracy: 0.9725
Epoch 9/11
764/764 [==============================] - 89s 116ms/step - loss: 1.3329 - accuracy: 0.9777
Epoch 10/11
764/764 [==============================] - 89s 116ms/step - l

##Evalutation

In [ ]:
audios_ev = []
time_ev = []
file_name = []
file = os.listdir('/content/evaluation/')
for name in file:
      file_name.append(name)
      lung = len(np.array(read('/content/evaluation/' + name)[1]))
      if lung == 11999:
        appo = np.array(read('/content/evaluation/' + name)[1])
      elif lung < 11999:
        p = 11999 - lung
        appo = np.pad(np.array(read('/content/evaluation/' + name)[1]), (0, p))
      time_ev.append(appo)
      audios_ev.append(np.abs(fft(appo)))

In [ ]:
audios_ev = np.array(audios_ev)
time_ev = np.array(time_ev)
audios_ev = ss1.transform(audios_ev)
audios_ev = audios_ev.reshape((5832, 11999, 1))
time_ev = sst1.transform(time_ev)
time_ev = time_ev.reshape((5832, 11999, 1))

In [ ]:
c = model.predict([audios_ev, time_ev])
c = c.argmax(axis=-1)
res = []
for val in c:
  res.append(letters[val])

In [ ]:
with open('result.csv', 'w') as file:
    file.write('Id,Predicted\n')
    for name,prediction in zip(file_name, res):
        name=name.replace(".wav",'')
        file.write(f"{name},{prediction}\n")